In [3]:
import os
os.environ['OPENAI_API_KEY'] = 'xxxxxxxxx'

In [8]:
from typing import List, Optional
from openai import OpenAI
from pydantic import BaseModel, ValidationInfo, field_validator, ValidationError, FieldValidationInfo, Field, model_validator
import instructor

client = instructor.patch(OpenAI())

class Validation(BaseModel):
    is_valid: bool
    error_messages: Optional[str] = Field(None, description="Error messages if any")


class Relation(BaseModel):
    head: str
    tail: str
    label: str


    @model_validator(mode="after")
    def validate_sources(self, info: FieldValidationInfo) -> "Relation":

        context = info.context.get("text", None)

        resp: Validation = client.chat.completions.create(
            response_model=Validation,
            messages=[
                {
                    "role": "user",
                    "content": f"Is the given relation label valid in the following context?\n\Relation: {self.head} -> {self.label} -> {self.tail}\n\nContext: {context}",
                }
            ],
            model="gpt-3.5-turbo",
        )

        if resp.is_valid:
            return self

        raise ValueError(resp.error_messages)


class RelationPrediction(BaseModel):
    # question: str
    relation: List[Relation]
    